In [1]:
from riotwatcher import TftWatcher, ApiError
import requests
from tqdm import tqdm 
import json

In [11]:
my_api = 'your api-key'
tft_watcher = TftWatcher(api_key=my_api)

In [3]:
# data extraction functions

# get match info by match id
# max 100 requests in 2 mins limited by riotwatcher lib
def get_matches_info(matches=[], region='RU'):
    matches_list = []
    errors = 0
    for match in tqdm(matches):
        try:
            match_1 = tft_watcher.match.by_id(region, match_id=match)
        except:
            errors += 1
            print('Some error occured. Sleeping for 1 min. Total errors -', errors)
            time.sleep(60)
            continue
        matches_list.append(match_1)
    return matches_list

# get challengers info from different regions
def get_challengers(region='RU'):
    return tft_watcher.league.challenger(region=region)

# get challengers puuid's from previous function data
def get_challengers_puuid(data=[], region='RU'):
    challengers = []
    for i in tqdm(data['entries']):
        try:
            challengers.append(tft_watcher.summoner.by_name(region=region, summoner_name=i['summonerName'])['puuid'])
        except:
            print('error')
            continue
    return challengers

# get matches by puuid
def get_matches_id_by_player_puuid(puuid_list=[], region='RU', matches_quantity=1):
    matches = []
    for puuid in tqdm(puuid_list):
        matches.append(tft_watcher.match.by_puuid(region=region, puuid=puuid, count=matches_quantity))
    return [item for sublist in matches for item in sublist]

In [ ]:
matches_quantity = 'select matches quantity for each challenger'

# these are all regions availiable
regions = ['BR1', 'EUN1', 'EUW1', 'JP1', 'KR', 'LA1', 'LA2', 'NA1', 'OC1', 'TR1', 'RU', 'PH2', 'SG2', 'TH2', 'TW2', 'VN2']

for region in regions:

    print(f'Getting list of challengers from {region} region')
    challs_list = get_challengers(region=region)

    print(f'Getting {region} challenger league puuids')
    challs_puuid = get_challengers_puuid(data=challs_list, region=region)

    print('Getting match ids')
    matches_id = set(get_matches_id_by_player_puuid(puuid_list=challs_puuid, region=region, matches_quantity=matches_quantity))

    print('Getting matches data')
    matches_data = get_matches_info(matches=matches_id, region=region)

    matches_data_filename = f'{region}_matches_data'
    matches_id_filename   = f'{region}_matches_id'

    print(f'Saving matches data to json. Total games extracted: {len(matches_data)}')
    with open(matches_data_filename + '.json', 'w') as fout:
        json.dump(matches_data , fout)

    print('Saving matches id to json')
    with open(matches_id_filename + '.json', 'w') as fout:
        json.dump(list(matches_id) , fout)
        
    del matches_data
    
    print(f'{region} region data gathered and saved')

In [ ]:
# optionally you can add region lable to each game and unite all files

matches_data_united = []
path = "path to folder with matches data files"
for filename in tqdm(os.listdir(path)):
    with open(path + filename, 'r') as r:
        file = json.load(r)
    region_dict = {'region':filename.split('_')[0]}
    for match in file:
        match.update(region_dict)
    print(region_dict['region'])
    matches_data_united.extend(file)

with open('path to save', 'w') as w:
    json.dump(matches_data_united, w)
    
# same to match ids files
matches_id_united = []
path = "path to folder with matches id files"
for filename in os.listdir(path):
    with open(path + filename, 'r') as r:
        file = json.load(r)
    matches_id_united.extend(file)
    
with open('path to save', 'w') as w:
    json.dump(matches_id_united, w)